In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [1]:
import pandas as pd
import xlrd
import csv

data_xls = pd.read_excel("/home/kali/Desktop/Data_OpelCorsa.xlsx", "Sheet2" , index_col=None)
data_xls.to_csv('Data_OpelCorsa.csv', encoding='utf-8')

df = pd.read_csv("Data_OpelCorsa.csv")
print(df)

      Unnamed: 0  fiyat        il              model   yil         yakit  \
0              0  24900  İstanbul     1.3 CDTI Enjoy  2004         Dizel   
1              1  42500  İstanbul     1.3 CDTI Enjoy  2008         Dizel   
2              2  63750  İstanbul          1.4 Enjoy  2017        Benzin   
3              3  25000  diger_il  1.3 CDTI Essentia  2004         Dizel   
4              4  29750  diger_il  1.3 CDTI Essentia  2011         Dizel   
...          ...    ...       ...                ...   ...           ...   
1478        1478  25500  diger_il  1.3 CDTI Essentia  2006         Dizel   
1479        1479  27750  diger_il  1.3 CDTI Essentia  2005         Dizel   
1480        1480  47500  İstanbul     1.3 CDTI Enjoy  2012         Dizel   
1481        1481  33500  diger_il       1.2 Essentia  2008        Benzin   
1482        1482  18750  diger_il          1.4 Swing  1997  Benzin & LPG   

         vites      km         kasa_tipi  motor_gucu_hp  motor_hacmi_cc  \
0       Manu

In [ ]:
import pandas as pd
import numpy as np
import xlrd
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

data_xls = pd.read_excel("/home/kali/Desktop/Data_OpelCorsa.xlsx", "Sheet2" , index_col=None)
data_xls.to_csv('Data_OpelCorsa.csv', encoding='utf-8')

df = pd.read_csv("Data_OpelCorsa.csv")

dummies = pd.get_dummies(df.yakit)
merged = pd.concat([df,dummies],axis="columns")
final = merged.drop(['yakit','Dizel', 'Unnamed: 0'],axis='columns')

dummies3 = pd.get_dummies(df.vites)
merged3 = pd.concat([final,dummies3],axis="columns")
final3 = merged3.drop(["vites","Manuel"],axis='columns')

dummies4 = pd.get_dummies(df.kasa_tipi)
merged4 = pd.concat([final3,dummies4],axis="columns")
final4 = merged4.drop(["kasa_tipi","Sedan"],axis='columns')

dummies6 = pd.get_dummies(df.il)
merged6 = pd.concat([final4,dummies6],axis="columns")
final6 = merged6.drop(["il","İstanbul"],axis='columns')

dummies7 = pd.get_dummies(df.model)
merged7 = pd.concat([final6,dummies7],axis="columns")
final7 = merged7.drop(["model","1.3 CDTI Enjoy"],axis='columns')

dummies8 = pd.get_dummies(df.renk)
merged8 = pd.concat([final7,dummies8],axis="columns")
final8 = merged8.drop(["renk","Beyaz"],axis='columns')

dummies9 = pd.get_dummies(df.durumu)
merged9 = pd.concat([final8,dummies9],axis="columns")
final9 = merged9.drop(["durumu","İkinci El"],axis='columns')

dummies10 = pd.get_dummies(df.cekis)
merged10 = pd.concat([final9,dummies10],axis="columns")
final10 = merged10.drop(["cekis","Önden"],axis='columns')

dummies11 = pd.get_dummies(df.garanti)
merged11 = pd.concat([final10,dummies11],axis="columns")
final11 = merged11.drop(["garanti","Evet"],axis='columns')

dummies12 = pd.get_dummies(df.kimden)
merged12 = pd.concat([final11,dummies12],axis="columns")
final12 = merged12.drop(["kimden","Sahibinden"],axis='columns')


#in order to prevent the error "Data with input dtype int64 was converted to float64 by MinMaxScaler".
final12 = final12.astype(float)

X = final12.iloc[:, 1:23].values
y = final12.iloc[:, 0].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#scaling x and y.
x_scaler=MinMaxScaler()
y_scaler=MinMaxScaler()

y_test=y_test.reshape(-1,1)
y_train=y_train.reshape(-1,1)
X_train= x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.transform(y_test)

def search(yil, km, motorgucu, motorhacmi, benzin, benzinvelpg, otomatik, hatchback3, hatchback5, digerIl, essentia12, essentia13,
           enjoy14, swing14, kirmizi, siyah, diger_renk, sifir, wd4, arkadan, hayir, galeriden):
    search = np.array([yil, km, motorgucu, motorhacmi, benzin, benzinvelpg, otomatik, hatchback3, hatchback5, digerIl, essentia12, essentia13,
                       enjoy14, swing14, kirmizi, siyah, diger_renk, sifir, wd4, arkadan, hayir, galeriden])
    #reshaping data.
    search=search.reshape(1,-1)
    #resut has been scaled according to x in order to obtain accurate result.
    result = x_scaler.transform(search)
    #return result to the user interface
    return result

#Initializing Neural Network
classifier = Sequential()

#Keras 2 API'ya göre düzenlendi/hazırlandı.
# Adding the input layer and the first hidden layer
classifier.add(Dense(kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 22, units = 4))
# Adding the second hidden layer
classifier.add(Dense(kernel_initializer = 'uniform', activation = 'sigmoid', units = 8))
# Adding the output layer
classifier.add(Dense(kernel_initializer = 'uniform', activation = 'sigmoid', units = 1))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['mse'])
# Fitting our model
classifier.fit(X_train, y_train, epochs = 750)
# Predicting the Test set results


y_pred = classifier.predict(X_train)
y_pred=y_scaler.inverse_transform(y_pred)
y_test=y_scaler.inverse_transform(y_test)
y_train=y_scaler.inverse_transform(y_train)

X_train=x_scaler.inverse_transform(X_train)
X_train= X_train.astype(int)

from sklearn.metrics import r2_score
print("R2: ",r2_score(y_train, y_pred))

#function which calculates the price of vehicles.
def predict(result):
    new_pred = classifier.predict(result)
    new_pred=y_scaler.inverse_transform(new_pred)
    result = x_scaler.inverse_transform(result)
    #Veride virgülden sonraki kısımları atıyoruz.
    result = result.astype(int)
    print("X=%s, Predicted=%s" % (result[0], new_pred[0]))
    new_pred = new_pred.astype(int)
    #Sending the predict to the interface
    return new_pred[0]

from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("Machine Learning")
        MainWindow.resize(776, 640)

        #variables defined.
        self.girilenil = 0
        self.girilenYil = 0
        self.girilenKm = 0
        self.girilenMotorgucu = 0
        self.girilenMotorhacmi = 0
        self.essentia12 = 0
        self.essentia13 = 0
        self.enjoy14 = 0
        self.swing14 = 0
        self.benzin = 0
        self.benzinveLPG = 0
        self.otomatik = 0
        self.hb3 = 0
        self.hb5 = 0
        self.wd4 = 0
        self.arkadan = 0
        self.kirmizi = 0
        self.siyah = 0
        self.diger_renk = 0
        self.hayir = 0
        self.galeriden = 0
        self.sifir = 0


        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.horizontalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget.setGeometry(QtCore.QRect(20, 20, 201, 80))
        self.horizontalLayoutWidget.setObjectName("horizontalLayoutWidget")
        self.horizontalLayout_3 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget)
        self.horizontalLayout_3.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_3.setObjectName("horizontalLayout_3")
        self.label = QtWidgets.QLabel(self.horizontalLayoutWidget)
        self.label.setObjectName("label")
        self.horizontalLayout_3.addWidget(self.label)
        self.il = QtWidgets.QLineEdit(self.horizontalLayoutWidget)
        self.il.setObjectName("il")
        self.horizontalLayout_3.addWidget(self.il)
        self.horizontalLayoutWidget_2 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_2.setGeometry(QtCore.QRect(20, 100, 201, 80))
        self.horizontalLayoutWidget_2.setObjectName("horizontalLayoutWidget_2")
        self.horizontalLayout_4 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_2)
        self.horizontalLayout_4.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_4.setObjectName("horizontalLayout_4")
        self.label_2 = QtWidgets.QLabel(self.horizontalLayoutWidget_2)
        self.label_2.setObjectName("label_2")
        self.horizontalLayout_4.addWidget(self.label_2)
        self.model = QtWidgets.QComboBox(self.horizontalLayoutWidget_2)
        self.model.setObjectName("model")
        self.model.addItem("")
        self.model.addItem("")
        self.model.addItem("")
        self.model.addItem("")
        self.model.addItem("")
        self.horizontalLayout_4.addWidget(self.model)
        self.horizontalLayoutWidget_3 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_3.setGeometry(QtCore.QRect(20, 180, 201, 80))
        self.horizontalLayoutWidget_3.setObjectName("horizontalLayoutWidget_3")
        self.horizontalLayout_5 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_3)
        self.horizontalLayout_5.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_5.setObjectName("horizontalLayout_5")
        self.label_3 = QtWidgets.QLabel(self.horizontalLayoutWidget_3)
        self.label_3.setObjectName("label_3")
        self.horizontalLayout_5.addWidget(self.label_3)
        self.yil = QtWidgets.QLineEdit(self.horizontalLayoutWidget_3)
        self.yil.setObjectName("yil")
        self.horizontalLayout_5.addWidget(self.yil)
        self.horizontalLayoutWidget_4 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_4.setGeometry(QtCore.QRect(20, 260, 201, 80))
        self.horizontalLayoutWidget_4.setObjectName("horizontalLayoutWidget_4")
        self.horizontalLayout_6 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_4)
        self.horizontalLayout_6.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_6.setObjectName("horizontalLayout_6")
        self.label_4 = QtWidgets.QLabel(self.horizontalLayoutWidget_4)
        self.label_4.setObjectName("label_4")
        self.horizontalLayout_6.addWidget(self.label_4)
        self.yakit = QtWidgets.QComboBox(self.horizontalLayoutWidget_4)
        self.yakit.setObjectName("yakit")
        self.yakit.addItem("")
        self.yakit.addItem("")
        self.yakit.addItem("")
        self.horizontalLayout_6.addWidget(self.yakit)
        self.horizontalLayoutWidget_5 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_5.setGeometry(QtCore.QRect(20, 340, 201, 80))
        self.horizontalLayoutWidget_5.setObjectName("horizontalLayoutWidget_5")
        self.horizontalLayout_7 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_5)
        self.horizontalLayout_7.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_7.setObjectName("horizontalLayout_7")
        self.label_5 = QtWidgets.QLabel(self.horizontalLayoutWidget_5)
        self.label_5.setObjectName("label_5")
        self.horizontalLayout_7.addWidget(self.label_5)
        self.vites = QtWidgets.QComboBox(self.horizontalLayoutWidget_5)
        self.vites.setObjectName("vites")
        self.vites.addItem("")
        self.vites.addItem("")
        self.horizontalLayout_7.addWidget(self.vites)
        self.horizontalLayoutWidget_6 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_6.setGeometry(QtCore.QRect(20, 420, 201, 80))
        self.horizontalLayoutWidget_6.setObjectName("horizontalLayoutWidget_6")
        self.horizontalLayout_8 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_6)
        self.horizontalLayout_8.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_8.setObjectName("horizontalLayout_8")
        self.label_6 = QtWidgets.QLabel(self.horizontalLayoutWidget_6)
        self.label_6.setObjectName("label_6")
        self.horizontalLayout_8.addWidget(self.label_6)
        self.kilometre = QtWidgets.QLineEdit(self.horizontalLayoutWidget_6)
        self.kilometre.setObjectName("kilometre")
        self.horizontalLayout_8.addWidget(self.kilometre)
        self.horizontalLayoutWidget_7 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_7.setGeometry(QtCore.QRect(20, 500, 201, 80))
        self.horizontalLayoutWidget_7.setObjectName("horizontalLayoutWidget_7")
        self.horizontalLayout_9 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_7)
        self.horizontalLayout_9.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_9.setObjectName("horizontalLayout_9")
        self.label_7 = QtWidgets.QLabel(self.horizontalLayoutWidget_7)
        self.label_7.setObjectName("label_7")
        self.horizontalLayout_9.addWidget(self.label_7)
        self.kasatipi = QtWidgets.QComboBox(self.horizontalLayoutWidget_7)
        self.kasatipi.setObjectName("kasatipi")
        self.kasatipi.addItem("")
        self.kasatipi.addItem("")
        self.kasatipi.addItem("")
        self.horizontalLayout_9.addWidget(self.kasatipi)
        self.horizontalLayoutWidget_8 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_8.setGeometry(QtCore.QRect(270, 20, 201, 80))
        self.horizontalLayoutWidget_8.setObjectName("horizontalLayoutWidget_8")
        self.horizontalLayout_10 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_8)
        self.horizontalLayout_10.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_10.setObjectName("horizontalLayout_10")
        self.label_8 = QtWidgets.QLabel(self.horizontalLayoutWidget_8)
        self.label_8.setObjectName("label_8")
        self.horizontalLayout_10.addWidget(self.label_8)
        self.motorgucu = QtWidgets.QLineEdit(self.horizontalLayoutWidget_8)
        self.motorgucu.setObjectName("motorgucu")
        self.horizontalLayout_10.addWidget(self.motorgucu)
        self.horizontalLayoutWidget_9 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_9.setGeometry(QtCore.QRect(270, 100, 201, 80))
        self.horizontalLayoutWidget_9.setObjectName("horizontalLayoutWidget_9")
        self.horizontalLayout_11 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_9)
        self.horizontalLayout_11.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_11.setObjectName("horizontalLayout_11")
        self.label_9 = QtWidgets.QLabel(self.horizontalLayoutWidget_9)
        self.label_9.setObjectName("label_9")
        self.horizontalLayout_11.addWidget(self.label_9)
        self.motorhacmi = QtWidgets.QLineEdit(self.horizontalLayoutWidget_9)
        self.motorhacmi.setObjectName("motorhacmi")
        self.horizontalLayout_11.addWidget(self.motorhacmi)
        self.horizontalLayoutWidget_10 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_10.setGeometry(QtCore.QRect(270, 180, 201, 80))
        self.horizontalLayoutWidget_10.setObjectName("horizontalLayoutWidget_10")
        self.horizontalLayout_12 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_10)
        self.horizontalLayout_12.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_12.setObjectName("horizontalLayout_12")
        self.label_10 = QtWidgets.QLabel(self.horizontalLayoutWidget_10)
        self.label_10.setObjectName("label_10")
        self.horizontalLayout_12.addWidget(self.label_10)
        self.cekis = QtWidgets.QComboBox(self.horizontalLayoutWidget_10)
        self.cekis.setObjectName("cekis")
        self.cekis.addItem("")
        self.cekis.addItem("")
        self.cekis.addItem("")
        self.horizontalLayout_12.addWidget(self.cekis)
        self.horizontalLayoutWidget_11 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_11.setGeometry(QtCore.QRect(270, 260, 201, 80))
        self.horizontalLayoutWidget_11.setObjectName("horizontalLayoutWidget_11")
        self.horizontalLayout_13 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_11)
        self.horizontalLayout_13.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_13.setObjectName("horizontalLayout_13")
        self.label_11 = QtWidgets.QLabel(self.horizontalLayoutWidget_11)
        self.label_11.setObjectName("label_11")
        self.horizontalLayout_13.addWidget(self.label_11)
        self.renk = QtWidgets.QComboBox(self.horizontalLayoutWidget_11)
        self.renk.setObjectName("renk")
        self.renk.addItem("")
        self.renk.addItem("")
        self.renk.addItem("")
        self.renk.addItem("")
        self.horizontalLayout_13.addWidget(self.renk)
        self.horizontalLayoutWidget_12 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_12.setGeometry(QtCore.QRect(270, 340, 201, 80))
        self.horizontalLayoutWidget_12.setObjectName("horizontalLayoutWidget_12")
        self.horizontalLayout_14 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_12)
        self.horizontalLayout_14.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_14.setObjectName("horizontalLayout_14")
        self.label_12 = QtWidgets.QLabel(self.horizontalLayoutWidget_12)
        self.label_12.setObjectName("label_12")
        self.horizontalLayout_14.addWidget(self.label_12)
        self.garanti = QtWidgets.QComboBox(self.horizontalLayoutWidget_12)
        self.garanti.setObjectName("garanti")
        self.garanti.addItem("")
        self.garanti.addItem("")
        self.horizontalLayout_14.addWidget(self.garanti)
        self.horizontalLayoutWidget_13 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_13.setGeometry(QtCore.QRect(270, 420, 201, 80))
        self.horizontalLayoutWidget_13.setObjectName("horizontalLayoutWidget_13")
        self.horizontalLayout_15 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_13)
        self.horizontalLayout_15.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_15.setObjectName("horizontalLayout_15")
        self.label_13 = QtWidgets.QLabel(self.horizontalLayoutWidget_13)
        self.label_13.setObjectName("label_13")
        self.horizontalLayout_15.addWidget(self.label_13)
        self.kimden = QtWidgets.QComboBox(self.horizontalLayoutWidget_13)
        self.kimden.setObjectName("kimden")
        self.kimden.addItem("")
        self.kimden.addItem("")
        self.horizontalLayout_15.addWidget(self.kimden)
        self.horizontalLayoutWidget_14 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_14.setGeometry(QtCore.QRect(270, 500, 201, 80))
        self.horizontalLayoutWidget_14.setObjectName("horizontalLayoutWidget_14")
        self.horizontalLayout_16 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_14)
        self.horizontalLayout_16.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_16.setObjectName("horizontalLayout_16")
        self.label_14 = QtWidgets.QLabel(self.horizontalLayoutWidget_14)
        self.label_14.setObjectName("label_14")
        self.horizontalLayout_16.addWidget(self.label_14)
        self.durumu = QtWidgets.QComboBox(self.horizontalLayoutWidget_14)
        self.durumu.setObjectName("durumu")
        self.durumu.addItem("")
        self.durumu.addItem("")
        self.horizontalLayout_16.addWidget(self.durumu)
        self.temizlebutton = QtWidgets.QPushButton(self.centralwidget)
        self.temizlebutton.setGeometry(QtCore.QRect(580, 40, 120, 40))
        self.temizlebutton.setObjectName("temizlebutton")
        self.hesaplabutton = QtWidgets.QPushButton(self.centralwidget)
        self.hesaplabutton.setGeometry(QtCore.QRect(580, 120, 120, 40))
        self.hesaplabutton.setObjectName("hesaplabutton")
        self.horizontalLayoutWidget_15 = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget_15.setGeometry(QtCore.QRect(570, 500, 160, 81))
        self.horizontalLayoutWidget_15.setObjectName("horizontalLayoutWidget_15")
        self.horizontalLayout_17 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_15)
        self.horizontalLayout_17.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_17.setObjectName("horizontalLayout_17")
        self.label_15 = QtWidgets.QLabel(self.horizontalLayoutWidget_15)
        self.label_15.setObjectName("label_15")
        self.horizontalLayout_17.addWidget(self.label_15)
        self.sonuc = QtWidgets.QLabel(self.horizontalLayoutWidget_15)
        self.sonuc.setText("")
        self.sonuc.setObjectName("sonuc")
        self.horizontalLayout_17.addWidget(self.sonuc)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 776, 22))
        self.menubar.setObjectName("menubar")
        self.menuGeli_mi = QtWidgets.QMenu(self.menubar)
        self.menuGeli_mi.setObjectName("menuGeli_mi")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.actionGrafik = QtWidgets.QAction(MainWindow)
        self.actionGrafik.setObjectName("actionGrafik")
        self.menuGeli_mi.addAction(self.actionGrafik)
        self.menubar.addAction(self.menuGeli_mi.menuAction())

        #only 4 digit number for year can be written.
        self.yil.setInputMask("0009")
        self.yil.setText("2000")
        self.yil.setCursorPosition(0)

        #only 7 digit number for km can be written.
        self.kilometre.setInputMask("0000009")
        self.kilometre.setText("1000000")
        self.kilometre.setCursorPosition(0)

        #only 4 digit engine power can be written.
        self.motorgucu.setInputMask("0009")
        self.motorgucu.setText("1000")
        self.motorgucu.setCursorPosition(0)

        #only 5 digit displacement can be written.
        self.motorhacmi.setInputMask("00009")
        self.motorhacmi.setText("99999")
        self.motorhacmi.setCursorPosition(0)

        #clicking Temizle button in order to clear data.
        self.temizlebutton.clicked.connect(self.temizle)
        #clicking Hesapla button in ordder to calculate the price. 
        self.hesaplabutton.clicked.connect(self.hesapla)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    #Fuction to turn default values of fields when clicking Temizle button.
    def temizle(self):
        self.il.clear()
        self.yil.setText("2000")
        self.kilometre.setText("1000000")
        self.motorgucu.setText("1000")
        self.motorhacmi.setText("99999")
        self.model.setCurrentIndex(0)
        self.yakit.setCurrentIndex(0)
        self.vites.setCurrentIndex(0)
        self.kasatipi.setCurrentIndex(0)
        self.cekis.setCurrentIndex(0)
        self.renk.setCurrentIndex(0)
        self.garanti.setCurrentIndex(0)
        self.kimden.setCurrentIndex(0)
        self.durumu.setCurrentIndex(0)

    #Funtion to calculate the price of vehicles when clicking Hesapla button.
    def hesapla(self):

        self.girilenYil = self.yil.text()
        self.girilenKm = self.kilometre.text()
        self.girilenMotorgucu = self.motorgucu.text()
        self.girilenMotorhacmi = self.motorhacmi.text()

        girilenIl = str(self.il.text())
        if girilenIl == "İstanbul" or girilenIl == "istanbul":
            self.girilenil = 0
        else:
            self.girilenil = 1

        self.secilenModel = int(self.model.currentIndex())
        if self.secilenModel == 0:
            self.essentia12 = 1
            self.essentia13 = 0
            self.enjoy14 = 0
            self.swing14 = 0
        elif self.secilenModel == 1:
            self.essentia12 = 0
            self.essentia13 = 0
            self.enjoy14 = 0
            self.swing14 = 0
        elif self.secilenModel== 2:
            self.essentia12 = 0
            self.essentia13 =1
            self.enjoy14 = 0
            self.swing14 = 0
        elif self.secilenModel ==3:
            self.essentia12 = 0
            self.essentia13 = 0
            self.enjoy14 = 1
            self.swing14 = 0
        elif self.secilenModel ==4:
            self.essentia12 = 0
            self.essentia13 = 0
            self.enjoy14 = 0
            self.swing14 = 1

        self.secilenYakit = int(self.yakit.currentIndex())
        if self.secilenYakit == 0:
            self.benzin = 1
            self.benzinveLPG = 0
        elif self.secilenYakit == 1:
            self.benzin = 0
            self.benzinveLPG = 1
        elif self.secilenYakit == 2:
            self.benzin = 0
            self.benzinveLPG = 0

        self.secilenVites = int(self.vites.currentIndex())
        if self.secilenVites == 0:
            self.otomatik = 0
        elif self.secilenVites == 1:
            self.otomatik = 1

        self.secilenKasaTipi = int(self.kasatipi.currentIndex())
        if self.secilenKasaTipi == 0:
            self.hb3 = 1
            self.hb5 = 0
        elif self.secilenKasaTipi == 1:
            self.hb3 = 0
            self.hb5 = 1
        elif self.secilenKasaTipi == 2:
            self.hb3 = 0
            self.hb5 = 0

        self.secilenCekis = int(self.cekis.currentIndex())
        if self.secilenCekis == 0:
            self.wd4 = 1
            self.arkadan = 0
        elif self.secilenCekis == 1:
            self.wd4 = 0
            self.arkadan = 1
        elif self.secilenCekis == 2:
            self.wd4 = 0
            self.arkadan = 0

        self.secilenRenk = int(self.renk.currentIndex())
        if self.secilenRenk == 0:
            self.kirmizi = 0
            self.siyah = 0
            self.diger_renk = 0
        if self.secilenRenk == 1:
            self.kirmizi = 1
            self.siyah = 0
            self.diger_renk = 0
        if self.secilenRenk == 2:
            self.kirmizi = 0
            self.siyah = 1
            self.diger_renk = 0
        if self.secilenRenk == 3:
            self.kirmizi = 0
            self.siyah = 0
            self.diger_renk = 1

        self.secilenGaranti = int(self.garanti.currentIndex())
        if self.secilenGaranti == 0:
            self.hayir = 0
        elif self.secilenGaranti == 1:
            self.hayir = 1

        self.secilenKimden = int(self.kimden.currentIndex())
        if self.secilenKimden == 0:
            self.galeriden = 1
        elif self.secilenKimden == 1:
            self.galeriden = 0

        self.secilenDurumu = int(self.durumu.currentIndex())
        if self.secilenDurumu == 0:
            self.sifir = 1
        elif self.secilenDurumu == 1:
            self.sifir = 0

        #Sending the data taken from interface to seaarch function in order to predict result.
        self.predict = search(self.girilenYil, self.girilenKm, self.girilenMotorgucu, self.girilenMotorhacmi, self.benzin, self.benzinveLPG, self.otomatik, self.hb3, self.hb5, self.girilenil,
                                             self.essentia12, self.essentia13, self.enjoy14, self.swing14, self.kirmizi, self.siyah, self.diger_renk, self.sifir, self.wd4, self.arkadan, self.hayir,
                                             self.galeriden)

        #writes the predicted result on the interface.
        self.sonuc.setText(str(predict(self.predict)))

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Machine Learning"))
        self.label.setText(_translate("MainWindow", "İl:"))
        self.label_2.setText(_translate("MainWindow", "Model:"))
        self.model.setItemText(0, _translate("MainWindow", "1.2 Essentia"))
        self.model.setItemText(1, _translate("MainWindow", "1.3 CDTI Enjoy"))
        self.model.setItemText(2, _translate("MainWindow", "1.3 CDTI Essentia"))
        self.model.setItemText(3, _translate("MainWindow", "1.4 Enjoy"))
        self.model.setItemText(4, _translate("MainWindow", "1.4 Swing"))
        self.label_3.setText(_translate("MainWindow", "YIl:"))
        self.label_4.setText(_translate("MainWindow", "Yakıt:"))
        self.yakit.setItemText(0, _translate("MainWindow", "Benzin"))
        self.yakit.setItemText(1, _translate("MainWindow", "Benzin & LPG"))
        self.yakit.setItemText(2, _translate("MainWindow", "Dizel"))
        self.label_5.setText(_translate("MainWindow", "Vites:"))
        self.vites.setItemText(0, _translate("MainWindow", "Manuel"))
        self.vites.setItemText(1, _translate("MainWindow", "Otomatik"))
        self.label_6.setText(_translate("MainWindow", "Km:"))
        self.label_7.setText(_translate("MainWindow", "Kasa Tipi:"))
        self.kasatipi.setItemText(0, _translate("MainWindow", "Hb 3 Kapı"))
        self.kasatipi.setItemText(1, _translate("MainWindow", "Hb 5 Kapı"))
        self.kasatipi.setItemText(2, _translate("MainWindow", "Sedan"))
        self.label_8.setText(_translate("MainWindow", "Motor Gücü:"))
        self.label_9.setText(_translate("MainWindow", "Motor Hacmi:"))
        self.label_10.setText(_translate("MainWindow", "Çekiş:"))
        self.cekis.setItemText(0, _translate("MainWindow", "4WD"))
        self.cekis.setItemText(1, _translate("MainWindow", "Arkadan"))
        self.cekis.setItemText(2, _translate("MainWindow", "Önden"))
        self.label_11.setText(_translate("MainWindow", "Renk:"))
        self.renk.setItemText(0, _translate("MainWindow", "Beyaz"))
        self.renk.setItemText(1, _translate("MainWindow", "Kırmızı"))
        self.renk.setItemText(2, _translate("MainWindow", "Siyah"))
        self.renk.setItemText(3, _translate("MainWindow", "Diğer renk"))
        self.label_12.setText(_translate("MainWindow", "Garanti:"))
        self.garanti.setItemText(0, _translate("MainWindow", "Evet"))
        self.garanti.setItemText(1, _translate("MainWindow", "Hayır"))
        self.label_13.setText(_translate("MainWindow", "Kimden:"))
        self.kimden.setItemText(0, _translate("MainWindow", "Galeriden"))
        self.kimden.setItemText(1, _translate("MainWindow", "Sahibinden"))
        self.label_14.setText(_translate("MainWindow", "Durumu:"))
        self.durumu.setItemText(0, _translate("MainWindow", "Sıfır"))
        self.durumu.setItemText(1, _translate("MainWindow", "İkinci el"))
        self.temizlebutton.setText(_translate("MainWindow", "Temizle"))
        self.hesaplabutton.setText(_translate("MainWindow", "Hesapla"))
        self.label_15.setText(_translate("MainWindow", "Sonuç:"))
        self.menuGeli_mi.setTitle(_translate("MainWindow", "Gelişmiş"))
        self.actionGrafik.setText(_translate("MainWindow", "Grafik"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())





Epoch 1/750
38/38 [==============================] - 1s 17ms/step - loss: 0.6862 - mse: 0.0477
Epoch 2/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6682 - mse: 0.0388
Epoch 3/750
38/38 [==============================] - 0s 4ms/step - loss: 0.6586 - mse: 0.0341
Epoch 4/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6549 - mse: 0.0323
Epoch 5/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6540 - mse: 0.0319
Epoch 6/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6537 - mse: 0.0318
Epoch 7/750
38/38 [==============================] - 0s 4ms/step - loss: 0.6534 - mse: 0.0316
Epoch 8/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6530 - mse: 0.0315
Epoch 9/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6527 - mse: 0.0313
Epoch 10/750
38/38 [==============================] - 0s 3ms/step - loss: 0.6523 - mse: 0.0311
Epoch 11/750
38/38 [==============================] - 0s 3

38/38 [==============================] - 0s 4ms/step - loss: 0.5949 - mse: 0.0057
Epoch 88/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5948 - mse: 0.0057
Epoch 89/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5948 - mse: 0.0056
Epoch 90/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5947 - mse: 0.0056
Epoch 91/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5945 - mse: 0.0055
Epoch 92/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5945 - mse: 0.0055
Epoch 93/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5944 - mse: 0.0054
Epoch 94/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5943 - mse: 0.0054
Epoch 95/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5942 - mse: 0.0054
Epoch 96/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5941 - mse: 0.0053
Epoch 97/750
38/38 [==============================] - 0s 3ms/st

38/38 [==============================] - 0s 3ms/step - loss: 0.5890 - mse: 0.0031
Epoch 173/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5890 - mse: 0.0031
Epoch 174/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5890 - mse: 0.0031
Epoch 175/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5889 - mse: 0.0031
Epoch 176/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5890 - mse: 0.0031
Epoch 177/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5889 - mse: 0.0030
Epoch 178/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5888 - mse: 0.0030
Epoch 179/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5888 - mse: 0.0030
Epoch 180/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5888 - mse: 0.0030
Epoch 181/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5887 - mse: 0.0030
Epoch 182/750
38/38 [==============================] -

38/38 [==============================] - 0s 3ms/step - loss: 0.5874 - mse: 0.0025
Epoch 258/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5874 - mse: 0.0025
Epoch 259/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5874 - mse: 0.0025
Epoch 260/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5873 - mse: 0.0024
Epoch 261/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0024
Epoch 262/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0025
Epoch 263/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0024
Epoch 264/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0024
Epoch 265/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0024
Epoch 266/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5873 - mse: 0.0024
Epoch 267/750
38/38 [==============================] -

38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 342/750
38/38 [==============================] - ETA: 0s - loss: 0.5860 - mse: 0.002 - 0s 3ms/step - loss: 0.5865 - mse: 0.0022
Epoch 343/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 344/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 345/750
38/38 [==============================] - 0s 4ms/step - loss: 0.5865 - mse: 0.0022
Epoch 346/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 347/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 348/750
38/38 [==============================] - 0s 4ms/step - loss: 0.5865 - mse: 0.0022
Epoch 349/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5866 - mse: 0.0022
Epoch 350/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5865 - mse: 0.0022
Epoch 351/750
38

38/38 [==============================] - 0s 4ms/step - loss: 0.5862 - mse: 0.0021
Epoch 427/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5861 - mse: 0.0021
Epoch 428/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5861 - mse: 0.0021
Epoch 429/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5862 - mse: 0.0021
Epoch 430/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5861 - mse: 0.0021
Epoch 431/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5862 - mse: 0.0021
Epoch 432/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5862 - mse: 0.0021
Epoch 433/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5861 - mse: 0.0021
Epoch 434/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5861 - mse: 0.0021
Epoch 435/750
38/38 [==============================] - 0s 4ms/step - loss: 0.5861 - mse: 0.0021
Epoch 436/750
38/38 [==============================] -

38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - mse: 0.0020
Epoch 512/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5859 - mse: 0.0020
Epoch 513/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - mse: 0.0020
Epoch 514/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5859 - mse: 0.0020
Epoch 515/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - mse: 0.0020
Epoch 516/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5859 - mse: 0.0020
Epoch 517/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - mse: 0.0020
Epoch 518/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - mse: 0.0020
Epoch 519/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5859 - mse: 0.0020
Epoch 520/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5859 - mse: 0.0020
Epoch 521/750
38/38 [==============================] -

38/38 [==============================] - 0s 3ms/step - loss: 0.5857 - mse: 0.0020
Epoch 597/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5858 - mse: 0.0020
Epoch 598/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5858 - mse: 0.0020
Epoch 599/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5858 - mse: 0.0020
Epoch 600/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5857 - mse: 0.0020
Epoch 601/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5857 - mse: 0.0020
Epoch 602/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5857 - mse: 0.0020
Epoch 603/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5858 - mse: 0.0020
Epoch 604/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5857 - mse: 0.0020
Epoch 605/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5857 - mse: 0.0020
Epoch 606/750
38/38 [==============================] -

38/38 [==============================] - 0s 2ms/step - loss: 0.5856 - mse: 0.0019
Epoch 682/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 683/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 684/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5857 - mse: 0.0020
Epoch 685/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 686/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5856 - mse: 0.0019
Epoch 687/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 688/750
38/38 [==============================] - 0s 2ms/step - loss: 0.5856 - mse: 0.0019
Epoch 689/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 690/750
38/38 [==============================] - 0s 3ms/step - loss: 0.5856 - mse: 0.0019
Epoch 691/750
38/38 [==============================] -